In [ ]:
import cv2
import numpy as np

In [ ]:
## ORB 알고리즘(Oriented fast and Rotated Brief)
import cv2 
import numpy as np


src = cv2.imread('image/rgb-1.jpg', 1)
src = cv2.resize(src, dsize = (0,0), fx = 0.5, fy = 0.5)

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

target = cv2.imread('image/ir-1.jpg', 0)


surf = cv2.xfeatures2d.SURF_create(upright = True)

kp1, des1 = surf.detectAndCompute(gray, None)
kp2, des2 = surf.detectAndCompute(target, None)



"""
bf = cv2.BFMatcher(cv2.NORM_HAMMING2, crossCheck=True)
matches = bf.match(des1,des2)
matches = sorted(matches, key = lambda x: x.distance)
"""



"""
for i in matches[:100]:
    idx_des1 = i.queryIdx
    x1, y1, =  kp1[idx_des1].pt
    
    idx_des2 = i.trainIdx
    x2, y2, =  kp2[idx_des2].pt
    
    cv2.circle(src, (int(x1), int(y1)), 3, (0, 0, 0), 3)
    cv2.circle(target, (int(x2), int(y2)), 3, (255, 0, 0), 3)
    
"""
src_draw = cv2.drawKeypoints(src, kp2, None,
                               flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
target_draw = cv2.drawKeypoints(target, kp2, None,
                               flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


cv2.imshow('target', target_draw)
cv2.imshow('src', src_draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
pip uninstall opencv-contrib-python

In [ ]:
pip uninstall opencv-python

In [ ]:
# 영상 불러오기
import sys

src1 = cv2.imread('image/rgb-1.jpg', cv2.IMREAD_GRAYSCALE)
src1 = cv2.resize(src1, dsize = (0,0), fx = 0.5, fy = 0.5)
src2 = cv2.imread('image/ir-1.jpg', cv2.IMREAD_GRAYSCALE)
src2 = cv2.resize(src2, dsize = (0,0), fx = 0.5, fy = 0.5)

if src1 is None or src2 is None:
    print('Image load failed!')
    sys.exit()

# 특징점 알고리즘 객체 생성 (KAZE, AKAZE, ORB 등)
#feature = cv2.KAZE_create() # 기본값인 L2놈 이용
#feature = cv2.AKAZE_create()
feature = cv2.ORB_create()

# 특징점 검출 및 기술자 계산
kp1, desc1 = feature.detectAndCompute(src1, None)
kp2, desc2 = feature.detectAndCompute(src2, None)

# 특징점 매칭
matcher = cv2.BFMatcher_create()
matches = matcher.match(desc1, desc2)

# 좋은 매칭 결과 선별
matches = sorted(matches, key=lambda x: x.distance)
good_matches = matches[:100]

print('# of kp1:', len(kp1))
print('# of kp2:', len(kp2))
print('# of matches:', len(matches))
print('# of good_matches:', len(good_matches))

# 호모그래피 계산
# DMatch 객체에서 queryIdx와 trainIdx를 받아와서 크기와 타입 변환하기
pts1 = np.array([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2).astype(np.float32)
pts2 = np.array([kp1[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2).astype(np.float32)
                
H, _ = cv2.findHomography(pts1, pts2, cv2.RANSAC) # pts1과 pts2의 행렬 주의 (N,1,2)

# 호모그래피를 이용하여 기준 영상 영역 표시
dst = cv2.drawMatches(src1, kp1, src2, kp2, good_matches, None,
                      flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

(h, w) = src1.shape[:2]

# 입력 영상의 모서리 4점 좌표
corners1 = np.array([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]
                    ).reshape(-1, 1, 2).astype(np.float32)

# 입력 영상에 호모그래피 H 행렬로 투시 변환
corners2 = cv2.perspectiveTransform(corners1, H)

# corners2는 입력 영상에 좌표가 표현되있으므로 입력영상의 넓이 만큼 쉬프트
corners2 = corners2 + np.float32([w, 0])

# 다각형 그리기
cv2.polylines(dst, [np.int32(corners2)], True, (0, 255, 0), 2, cv2.LINE_AA)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
target = cv2.imread('image/ir-1.jpg', 0)

dst = cv2.bilateralFilter(dst, -1, 10, 5)

cv2.imshow('target', target)
cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()